### MusicGen-Small Model

MusicGen models are trained on 30-second chunks of audio but it is possible to generate longer sequences with a simple windowing approach. Let’s use a fixed 30-second windows and slide the window by chunks of 10 seconds, keeping the last 20 seconds that were generated as context, to generate those 2 minute-long tracks.

In [1]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration
import numpy as np
import torch
import scipy
from IPython.display import Audio
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

sampling_rate = model.config.audio_encoder.sampling_rate
prompt = "rock and roll, active, dance-style music, elvis presley"

def generation(prompt, seconds):
    generation_len_10secs = 1
    audio_chunk = None
    generated_ids_concat = None
    prev_generated_ids = None
    # input 10 seconds if there is
    inputs = processor(#[prev_generated_ids[0,0].cpu().numpy()] if prev_generated_ids != None else None,
            text=[prompt],
            padding=True,
            return_tensors="pt",
            sampling_rate=sampling_rate
    )
        # output: putted 10 seconds as audio=[], and generated 10 seconds
    return model.generate(**inputs, max_new_tokens=1024)

generated_ids = generation(prompt, seconds=10)
scipy.io.wavfile.write("musicgen_out.wav", rate=sampling_rate, data=generated_ids[0, 0].numpy())
display(Audio(generated_ids[0].numpy(), rate=sampling_rate))

/Users/vladmac/Code/dl/music-lm-train/.env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/vladmac/Code/dl/music-lm-train/.env/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/Users/vladmac/Code/dl/music-lm-train/.env/lib/python3.12/site-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - s

IndexError: index out of range in self